In [3]:
from collections import namedtuple

Location = namedtuple( "Location", ["s", "lane"])

Lane = namedtuple("Lane", ["id", "number"]) # 0 for number means closest to center yellow line

Goal = Location



# class Map():
#     def __init__(s, lane)

STATES = [
    "ready_to_drive",
    "follow_line",
    "lane_change_left",
    "lane_change_right",
    "track_gap_left", # used to prepare for a lane change left
    "track_gap_right"
]


ALLOWED_TRANSITIONS = [
    [0, 1, 0, 0, 0, 0],
    [0, 1, 1, 1, 1, 1],
    [0, 1, 1, 0, 0, 0],
    [0, 1, 0, 1, 0, 0],
    [0, 1, 1, 0, 1, 0],
    []
]


    
    
    

TypeError: namedtuple() takes at least 2 arguments (1 given)

In [1]:
# NOTE - need to emphasize the weaknesses of cost function approach and
#        cost function design.
#      - may want to send desired vehicle state to trajectory
#      - we had a long discussion about whether or not to include "track gap"
#        state. There is good pedagogical value to the debate that happened.
#        Maybe Benjamin plays the role of "screw it! Add another state!" and
#        Toby wants to keep the design of the state machine simple.

ILLEGAL_COST = 100
DANGEROUS_COST = 1000

def decide_behavior(MAP, loc, predictions, goal):
    """
    - To be called every 200 - 400 ms.
    - We are NOT responsible for safety. That's the job of trajectory 
      (operating at a 50 ms update rate.)
    """
    if not car.initialized:
        car.state = "ready_to_drive"
    s,v,a,lane = process_localization_data(loc, MAP)
    possible_next_states = get_possible_states(car.state)
    costs = []
    for state in possible_next_states:
        cost = calculate_cost(state, s, v, a, lane, predictions, goal, MAP)
        costs.append((cost,state))
    _, next_state = min(costs)
    return next_state

def calculate_cost(state, s, v, a, lane, predictions, goal, MAP):
    # cost functions generally designed to output numbers between -1 and 1
    # unless dangerous or illegal behavior
    cost_functions = [
        lane_change_cost,
        velocity_cost,
        gap_size_cost
    ]
    weights = [
        1,
        1,
        1
    ]
    cost = 0
    for cost_func, weight in zip(cost_functions, weights):
        cost += weight * cost_func(state, s,v,a,lane,predictions,goal, MAP)
    return cost

def lane_change_cost(state, s,v,a,lane,predictions,goal, MAP):
    if state not in ["lane_change_left", "lane_change_right"]:
        return 0
    diffs = {"lane_change_left":-1, "lane_change_right":+1}
    next_lane = lane + diffs[state]
    if not MAP.lane_exists_at_location(s, next_lane):
        return DANGEROUS_COST
    if abs(goal.lane - next_lane) < abs(goal.lane - lane):
        return -1
    else:
        return 1

def velocity_cost(state, s,v,a,lane,predictions,goal, MAP):
    speed_limit = MAP.get_speed_limit(s, lane)
    velocity_for_state = get_velocity_for_state(state, s,v,a,lane,predictions,goal,MAP)
    if velocity_for_state > speed_limit: 
        return ILLEGAL_COST
    return (speed_limit - velocity_for_state) / speed_limit # between 0 and 1

def gap_size_cost(state, s,v,a,lane,predictions,goal, MAP):
    L = MAP.vehicle_length
    
    # next evaluate the cost associated with all the available transitions
    
    # COSTS INCLUDE: 
    # 1. Does this make progress to the goal 
    #    in terms of number of lane changes remaining. 
    #    Idea is to penalize lane changes if they aren't necessary.
    # 2. Velocity: We want to drive as close to the speed 
    #    limit as possible. (NOTE: Include speed limit in map)
    # 3. Gap size: when making lane changes we prefer to aim for 
    #    larger gaps than smaller ones.
    # 4. If you wanted to go further you might need a  
    

SyntaxError: invalid syntax (<ipython-input-1-59e3c9c7ecf0>, line 49)